In [27]:
%%html
<form action="javascript:code_disp()">
<input type="submit" id="button" value="Show code">
</form>

<script type="text/javascript">
function code_disp() {
    if (code_vis == true) {
        $('div.input').hide('500');
        $('#button').val('Show code');
    }
    else if (code_vis == false){
        $('div.input').show('500');
        $('#button').val('Hide code');
    }
    code_vis = !code_vis;
}

$(document).ready(
    function() {
        code_vis = false;
        $('div.input').hide();
    }
)
</script>

/
<div class="intro" style="width: 100%;">
<a href="https://sbhse.engineering.asu.edu">
<img style="position: absolute; left:42%; top: 10px; z-index: 0;" class="logo" src="https://sbhse.engineering.asu.edu/wp-content/uploads/2015/09/default-logo.png" alt="ASU &#8226; SBSE Logo" width="150" height="150"> 
</a>
<br>
<a href="http://www.public.asu.edu/~jmuthus/lab/">
<img style="position: relative; left:-65px; top: 55px;" class="logo" src="http://www.public.asu.edu/~jmuthus/lab/images/c_name_1.jpg" alt="ASU &#8226; Neural Microsys Logo" width="350" height="250">
</a>
<div class="line" style="width: 100%; height: 2px; position: absolute; left: 0px; top: 170px; background-color:#f4a442;">
</div>
<br><br><br><br><br>
<b style="text-align: left; text-indent: 0px; font-family: Arial, Helvetica, sans-serif; margin-left: 20px;"><h2>Peak Force data for different speeds: Cells from 11/20/17, 11/29/17, 12/3/17 </h2><br>
</div>

In [5]:
import os
from pprint import pprint
import re as regex

# === Gathers relevant filepaths ===
filepaths = []
filepaths_cleaned = []
filepaths_small = []

for dirname, dirnames, filenames in os.walk('.'):
    # print path to all subdirectories first.
    for subdirname in dirnames:
        filepaths.append(os.path.join(dirname, subdirname))

    # print path to all filenames.
    for filename in filenames:
        filepaths.append(os.path.join(dirname, filename))

      
for path in filepaths:
    if path[0:5] == "./exp" and path[0:11] != "./exp_11-24" and path[-3:] == 'XLS':
        filepaths_cleaned.append(path[2:])

for path in filepaths_cleaned:
    if regex.findall('11-20-17', path):
        filepaths_small.append(path)
filepaths_smaller = np.array_split(np.array(filepaths_small), 2)
# print(filepaths_cleaned)
# print("")
# pprint(filepaths_small)
# print(len(filepaths_small))
display(HTML('<h3>Tot Number of files: ' + str(len(filepaths_cleaned)) + '</h3>') )

In [4]:
import os
import pandas as pd
import re as regex
import xlrd
import xlwt
import numpy as np
import datetime
import time
import plotly.offline

from xlrd.sheet import ctype_text 
from openpyxl import load_workbook
from openpyxl.workbook import Workbook as openpyxlWorkbook
from collections import OrderedDict
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.core.display import display, HTML
from itertools import groupby
from pprint import pprint
# === End of lib imports === 

init_notebook_mode(connected=True)  # initializes plotly plots

# === Gathers relevant filepaths ===
filepaths = []
filepaths_cleaned = []
filepaths_small = []

for dirname, dirnames, filenames in os.walk('.'):
    # print path to all subdirectories first.
    for subdirname in dirnames:
        filepaths.append(os.path.join(dirname, subdirname))

    # print path to all filenames.
    for filename in filenames:
        filepaths.append(os.path.join(dirname, filename))

      
for path in filepaths:
    if path[0:5] == "./exp" and path[0:11] != "./exp_11-24" and path[-3:] == 'XLS':
        filepaths_cleaned.append(path[2:])
        
for path in filepaths_cleaned:
    if regex.findall('11-20-17', path):
        filepaths_small.append(path)
# ====================================

# === Mimics MATLAB Smooth Func ======
def smooth(a,WSZ):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)
    start = np.cumsum(a[:WSZ-1])[::2]/r
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

# ===================================


# === Puts excel data into dataframe hash table and obtains peak force for each ====
dataframe_collection = {}
peak_forces = []

filepaths_smaller = np.array_split(np.array(filepaths_small), 2)

def find_peak(array):
        array = np.array_split(array, 2)
        if (array[1][0] < min(array[0])):
            return min(array[1])
        else:
            return min(array[0])
        
for path in filepaths_cleaned:
    xl_file = pd.ExcelFile(path)
    dataframe_collection[path] = pd.DataFrame(xl_file.parse(sheet_name=0))
#     dataframe[i] = pd.read_excel(open('master.xlsx','rb'), sheet_name=0)
    dataframe_collection[path] = dataframe_collection[path].fillna('')
    list1 = dataframe_collection[path]['Time']
    list1 = np.array(list1)
    list1 = (list1 - list1[0])/1000000  # shift and div by 1E6
    list2 = dataframe_collection[path]['Tracking Value'].tolist()
    list2 = np.array(list2)*(9.8/.001) # mult force by 9.8 (to get in mg), div by .001 (to get in uN)
    list2 = np.array(list2) - list2[0] # normalize force (sub all vals from init)
    list2 = smooth(list2,71)
        
    peak_forces.append(find_peak(list2))
    
    trace1 = plotly.graph_objs.Scatter(
    x = list1,
    y = list2,
    mode='markers',
    #     error_y=dict(
    #         type='data',
    #         array=error_array,
    # #         value=0.5,
    #         symmetric=True,
    #         visible=True,
    #         color='#958c8d8',
    #         thickness='2',
    #         opacity=0.3
    #     )
    )

    layout = dict(
        title = 'Force vs Time' + " (" + path[0:len(path)-4] + ")",
        yaxis = dict(title = u'Force (\u03bc N)', zeroline = True),
        xaxis = dict(title = 'Time (s)', zeroline = True),
    )

    data =[
        trace1
    ]

    fig = dict(data=data, layout=layout)

#     plotly.offline.iplot(fig)

# display(HTML('<h3>Files: ' + str(filepaths_smaller) + '</h3>') )

points_tbl = {'Peak Force':peak_forces, 'Filename':filepaths_cleaned}
dataframe2 = pd.DataFrame.from_dict(points_tbl)
dataframe2
# pprint(filepaths_smaller[0])
# display(HTML('<h3>Peak forces: ' + str(peak_forces) + ' \u03bcN' + '</h3>') )

,Filename,Peak Force (μN)
0,exp_12-3-17/cell2_15ups.XLS,-39.788853
1,exp_12-3-17/cell4_90ups.XLS,-172.820655
2,exp_12-3-17/cell5_115ups.XLS,-202.610418
3,exp_12-3-17/cell4_115ups.XLS,-172.969917
4,exp_12-3-17/cell5_50ups.XLS,-158.435517
5,exp_12-3-17/cell4_33p5ups.XLS,-151.304351
6,exp_12-3-17/cell4_90ups_run2.XLS,-167.245723
7,exp_12-3-17/cell5_72ups.XLS,-148.794258
8,exp_12-3-17/cell5_5ups.XLS,-125.197688
9,exp_12-3-17/cell4_50ups_Run2.XLS,-153.081853
